# Wikipedia Pre-learnt Embedding

In [2]:
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
vocab = KeyedVectors.load_word2vec_format('../data/enwiki_20180420_100d.txt', binary = False)

In [2]:
#vocab.most_similar("card")
#vocab.similarity('bratislava','city')

In [4]:
texts_array = pd.read_pickle("../notebooks/texts_lemma_final.pkl") #returns list 

In [5]:
texts_array

['we would like to inform you that a per airline agent there be number possible alternative flight for your return flight the last option be to cancel your return flight they have advise that you can still take the flight from city city even you have cancel your return flight from city city to apply for a refund and or cancel your return flight visit your application and fill in the refund application form that you can find at the bottom of the page we will cancel the reservation regardless of the refundable amount the total amount will be then sent back to you use your prefer method of payment the process might take up to day refund still depends on the airline policy thank you for your understand if you have any question try our help centre or contact our nonstop customer support team',
 'upon check your payment be decline by our third party security provider and you be already refund day ago the fund might take work day to appear on your account your understand and patience be highl

In [45]:
import spacy
#from spacy import displacy
from collections import Counter
import en_core_web_md
nlp = en_core_web_md.load()

In [47]:
for index in range(len(texts_array)):
    if re.search("ho chi minh city", texts_array[index]):
        print(index)
        texts_array[index] = re.sub("ho chi minh city","city",texts_array[index])

517


In [48]:
def replace_names(texts,not_name_list):
    converted = []
    for index,text in enumerate(texts):
        doc = nlp(text)
        for e in reversed(doc.ents):#reversed to not modify the offsets of other entities when substituting
            if(e.label_ == "PERSON"):
                if e.text in not_name_list:
                    continue
                print(index,e)
                start = e.start_char
                end = start + len(e.text)
                text = text[:start] + "name" + text[end:]
        converted.append(text)
    return converted

#not_name = ["judy city",""]
not_name = ["kg","airways","puerto princessa","zakinthos","kilograms", "kilogram"]
texts_array2 = replace_names(texts_array,not_name)

8 sayeed
31 romy
33 ryan air
47 matias klaus
47 maria klaus
48 puerto plata
63 trustpilot
82 jaqueline
94 randy derochies
94 hunter luther
94 randy derochie
117 raue
117 wilson
132 john wayne city city city
132 thu dec
134 annabelle
135 karl
138 prabhu
150 john m
150 john
163 ingram
176 thea
203 bernadeth g
213 lynch
215 sjoers hulten
226 judy
226 linda renstrљm
230 valbona hamiti
236 clare
240 dec fr
240 dec ibiza
263 reliford
304 m karnphian
309 albert m
309 trustpilot
309 albert
319 von b i
335 denpasar dili
336 kathlene
339 richard charles knapp
341 gubaton
358 vincent
371 maryjane v
375 guy dorenstein
381 alvin
388 nizhnevartovsk
388 nizhnevartovsk
404 haripersad
405 jayceon
424 vega
424 vega
443 kinabalu
444 perdicchia
453 martin
460 leticia
460 leticia
460 leticia
477 raleigh durham
511 igor
524 stavroula kontomichaloy
528 bremvag
547 nizar hadidi
559 easilypay
583 malay patel
583 m patel
597 m anagnostou
613 toдenovskщ
624 anna tsybulska
634 huseyin guven
634 m guven
644 m seub

In [49]:
#small edits
import re
for index,text in enumerate(texts_array2):
    texts_array2[index] = re.sub("citycity","city city",texts_array2[index])
    texts_array2[index] = re.sub("currencyfor","currency for",texts_array2[index])
    texts_array2[index] = re.sub("credit-card","credit card",texts_array2[index])
    texts_array2[index] = re.sub("currencyaccording","currency according",texts_array2[index])
    texts_array2[index] = re.sub("currencyatlanta","currency atlanta",texts_array2[index])
    texts_array2[index] = re.sub("currencytotal","currency total",texts_array2[index])
    texts_array2[index] = re.sub("easilypay","easily pay",texts_array2[index])
    texts_array2[index] = re.sub("immediatelyinform","immediately inform",texts_array2[index])
    texts_array2[index] = re.sub("currencyvoucher","currency voucher",texts_array2[index])
    texts_array2[index] = re.sub("delayin","delay in",texts_array2[index])

In [50]:
from nltk import word_tokenize
def tokenize_dataset(list_of_texts):
    word_tokens = []
    for i in range(len(list_of_texts)):
        word_tokens.append(word_tokenize(list_of_texts[i]))
    return word_tokens
    
def get_text_lengths(dataset_list):
    text_lengths = []
    for text in range(0,len(dataset_list)):
        text_lengths.append(len(dataset_list[text]))
    return text_lengths

word_tokens = tokenize_dataset(texts_array2)
lengths = pd.DataFrame(get_text_lengths(word_tokens))
lengths.rename(columns = {0:"text"},inplace = True)

In [51]:
lengths.describe()

,text
count,719.000000
mean,115.688456
std,55.481436
min,3.000000
25%,77.000000
50%,105.000000
75%,144.000000
max,397.000000


In [ ]:
#incase of imdb 
#take only those shorter or equal to 179
indices = lengths[lengths.text < 180].index
texts_array = pd.DataFrame(texts_array)
texts_array = texts_array.iloc[indices,:]
word_tokens = [text for text in word_tokens if len(text)<180]

max_text_length = 179
embedding_dim = 100
texts_holder = np.zeros([texts_array.shape[0],max_text_length,embedding_dim]) #401 needs to be checked
texts_array.columns = ["text"]

In [52]:
targets = pd.read_pickle("../texts_for_benchmarking.pkl")["EMPATHY_RECODED"]
targets = pd.DataFrame(targets)
neg_in = targets[targets["EMPATHY_RECODED"] == 0].index
pos_in = targets[targets["EMPATHY_RECODED"] == 1].index

In [53]:
max_text_length = 397
embedding_dim = 100
texts_holder = np.zeros([719,max_text_length,embedding_dim])

In [54]:
skip_words = ["bogot","joveth","ozty","eurwings","zgkb","medell","denpasar-bali","port-au-prince","sjoers","cancњn",
             "renstrљm","contactar","karnphian","josћ","eurpean","hyvљ","relaci","gubaton","mhdsnt","dorenstein",
             "canotic","haripersad","bangkik","inconviences","wietjet","perdicchia","occured","refunf","omanair",
             "wowair","areoflot","st.","bremvag","oyicxw","memmigen","kateюina","toдenovskщ","tsybulska",
              "atambua-timor","(","morhental","qeenstown","palachovo","el-sheikh","aћreas","derochies","nykoing",
              "ettљ","updrade","aiways","nchatz"]


#notaffect problem

#treba upravit v povodnom regexe

for text in range(0,719):
    new_word = 0
    for word in range(0,max_text_length-1):
        if word == lengths.iloc[text,0]:
            break
        if word_tokens[text][word] in skip_words:
            break
        texts_holder[text,new_word] = vocab[word_tokens[text][word]] #otestuj ako presne to fungije s tou dimenziou
        new_word += 1
    print("text",text,"complete")
        

text 0 complete
text 1 complete
text 2 complete
text 3 complete
text 4 complete
text 5 complete
text 6 complete
text 7 complete
text 8 complete
text 9 complete
text 10 complete
text 11 complete
text 12 complete
text 13 complete
text 14 complete
text 15 complete
text 16 complete
text 17 complete
text 18 complete
text 19 complete
text 20 complete
text 21 complete
text 22 complete
text 23 complete
text 24 complete
text 25 complete
text 26 complete
text 27 complete
text 28 complete
text 29 complete
text 30 complete
text 31 complete
text 32 complete
text 33 complete
text 34 complete
text 35 complete
text 36 complete
text 37 complete
text 38 complete
text 39 complete
text 40 complete
text 41 complete
text 42 complete
text 43 complete
text 44 complete
text 45 complete
text 46 complete
text 47 complete
text 48 complete
text 49 complete
text 50 complete
text 51 complete
text 52 complete
text 53 complete
text 54 complete
text 55 complete
text 56 complete
text 57 complete
text 58 complete
text 59

text 488 complete
text 489 complete
text 490 complete
text 491 complete
text 492 complete
text 493 complete
text 494 complete
text 495 complete
text 496 complete
text 497 complete
text 498 complete
text 499 complete
text 500 complete
text 501 complete
text 502 complete
text 503 complete
text 504 complete
text 505 complete
text 506 complete
text 507 complete
text 508 complete
text 509 complete
text 510 complete
text 511 complete
text 512 complete
text 513 complete
text 514 complete
text 515 complete
text 516 complete
text 517 complete
text 518 complete
text 519 complete
text 520 complete
text 521 complete
text 522 complete
text 523 complete
text 524 complete
text 525 complete
text 526 complete
text 527 complete
text 528 complete
text 529 complete
text 530 complete
text 531 complete
text 532 complete
text 533 complete
text 534 complete
text 535 complete
text 536 complete
text 537 complete
text 538 complete
text 539 complete
text 540 complete
text 541 complete
text 542 complete
text 543 c

In [55]:
import pickle
filename = '../processed_final_texts/wiki_embedded_texts_lemma.pkl'
outfile = open(filename,'wb')
pickle.dump(texts_holder,outfile)
outfile.close()

In [56]:
pos_texts = texts_holder[pos_in]#.astype("float32")
neg_texts = texts_holder[neg_in]#.astype("float32")
np.save("../processed_final_texts/empathy_pos_lemma_f_64.npy",pos_texts)
np.save("../processed_final_texts/empathy_neg_lemma_f_64.npy",neg_texts)

# Sentiment embedding

In [58]:
import pickle 
#load from pickle
corpora = pickle.load(open("../sentiment_emb.pkl","rb"))
max_text_length = 397
embedding_dim = 50

skip_words = ["bogot","joveth","ozty","eurwings","zgkb","medell","denpasar-bali","port-au-prince","sjoers","cancњn",
             "renstrљm","contactar","karnphian","josћ","eurpean","hyvљ","relaci","gubaton","mhdsnt","dorenstein",
             "canotic","haripersad","bangkik","inconviences","wietjet","perdicchia","occured","refunf","omanair",
             "wowair","areoflot","st.","bremvag","oyicxw","memmigen","kateюina","toдenovskщ","tsybulska",
              "atambua-timor","(","morhental","qeenstown","palachovo","el-sheikh","aћreas","derochies","nykoing",
              "ettљ","updrade","aiways","nchatz",
             "refundable","nonrefundable","ljubljana"] #new corpora specific

texts_holder = np.zeros([719,max_text_length,embedding_dim])

unknown = []

for text in range(0,719):
    new_word = 0
    for word in range(0,max_text_length-1):
        if word == lengths.iloc[text,0]:
            break
        try:
            texts_holder[text,new_word] = corpora[word_tokens[text][word]] #otestuj ako presne to fungije s tou dimenziou
        except:
            print(text,word_tokens[text][word])
            unknown.append(word_tokens[text][word])
        new_word += 1
    print("text",text,"complete")




0 refundable
text 0 complete
text 1 complete
text 2 complete
3 refundable
3 refundable
3 refundable
text 3 complete
4 nonrefundable
4 refundable
4 refundable
4 refundable
text 4 complete
text 5 complete
6 refundable
6 waive
text 6 complete
text 7 complete
text 8 complete
text 9 complete
text 10 complete
11 trustpilot
11 trustpilot
text 11 complete
text 12 complete
text 13 complete
text 14 complete
text 15 complete
text 16 complete
text 17 complete
text 18 complete
19 credential
19 dissemination
19 prohibit
text 19 complete
text 20 complete
text 21 complete
text 22 complete
text 23 complete
text 24 complete
25 lublin
text 25 complete
text 26 complete
text 27 complete
text 28 complete
text 29 complete
text 30 complete
31 trustpilot
text 31 complete
text 32 complete
33 с
text 33 complete
34 hotlines
text 34 complete
text 35 complete
text 36 complete
text 37 complete
text 38 complete
text 39 complete
text 40 complete
41 с
41 refundable
41 с
41 refundable
41 refundable
41 refundable
41 refu

366 refundable
text 366 complete
text 367 complete
text 368 complete
text 369 complete
text 370 complete
371 trustpilot
text 371 complete
text 372 complete
text 373 complete
374 refundable
374 hotlines
text 374 complete
text 375 complete
376 iata
text 376 complete
text 377 complete
378 refundable
378 refundable
text 378 complete
379 refundable
text 379 complete
380 additionally
text 380 complete
381 trustpilot
text 381 complete
text 382 complete
text 383 complete
384 expedite
text 384 complete
text 385 complete
386 qz
386 qz
text 386 complete
text 387 complete
388 nizhnevartovsk
388 ural
388 refundable
text 388 complete
389 airway
389 brokerage
389 accordance
text 389 complete
text 390 complete
391 marrakesh
391 marrakesh
text 391 complete
text 392 complete
text 393 complete
text 394 complete
395 с
text 395 complete
text 396 complete
397 entitle
text 397 complete
398 refundable
text 398 complete
text 399 complete
text 400 complete
text 401 complete
text 402 complete
text 403 complete
t

In [60]:
len(set(unknown))

110

In [62]:
pos_texts = texts_holder[pos_in]#.astype("float32")
neg_texts = texts_holder[neg_in]#.astype("float32")
np.save("../processed_final_texts/empathy_pos_lemma_f_sent.npy",pos_texts)
np.save("../processed_final_texts/empathy_neg_lemma_f_sent.npy",neg_texts)